# Molecular dating of Holocephali using fossil calibrations


## 1. Introduction

### 1.1 L'article
Grâce à la leur position à la base des poissons __osseux__, les poissons __cartilagineux__ sont considérés comme un excellent _outgroup_ dans les études comparatives des vertebrés.
<br/>
Récemment, le génome de _Callorhinchus milii_ (__chimère éléphant__) a été proposé comme génome modèle pour comprendre l'histoire évolutive des génomes vertebrés grâce sa position phylogénétique et à son génome de petite taille (910 Mb). Malgré un tel intérêt croissant, l’histoire évolutive des _holophaceans_ modernes ainsi que leur rélation avec _Elasmobranchii_ (sous-classe de _Chondrichthyes_, qui regroupe les requins, raies et chimères) et d'autres vertebrés à machoire n'a pas été bien documentée à cause de l'absence de matériaux fossiles bien conservés.
<br/><br/>
Dans cette étude, les auteurs ont assemblé le génome mitchondrial de 8 représentants de trois familles d'_holophaceans_ modernes, pour pouvoir étudier leurs rélations phylogénétiques et histoire évolutive. Pour faire cela, ils ont analysé par __maximum__ de __vraisemblance__ les séquences alignées sans ambiguïté de ces _holophaceans_ avec 17 autres vertebrés (9,409 positions nucléotidiques en excludant la troisième position du codon).
<br/>
L'arbre généré supporte l'hypothèse d'une seule origine des _holophaceans_ modernes et leur relation avec le groupe-soeur des _Elasmobranchii_. 
L’arbre mitogénomique a récupéré le plus grand nombre de _Callorhinchus_ basaux dans les _Chimaeriformes_, lequel est groupe-soeur avec les clades des deux familles restantes (_Rhinochimaeridae_ et _Chimaeridae_). 
<br/>
L’arbre temporel dérivé d’une horloge moléculaire détendue bayésienne suggère que les _holophaceans_ sont originaires du __Silurien__ vers 420 Ma, ayant survécu dans la fin du __Permien__ à l'extinction de masse et qui ont subi une diversification familiale de la fin du __Jurassique__ au début du __Crétacé__. Ce scénario évolutif proposé est bien en accord avec celui basé sur les observations paléontologiques.
<br/><br/>
Source: [Evolutionary Origin and Phylogeny of the Modern Holocephalans (Chondrichthyes: Chimaeriformes): A Mitogenomic Perspective](https://academic.oup.com/mbe/article/27/11/2576/1121974?login=true)

### 1.2 Le projet
La datation d’une phylogénie repose sur l’interprétation de la divergence entre les séquences à la lumière des calibrations fossiles, et à la lumière des _a priori_ sur ces calibrations et sur la forme de la phylogénie. <br/>
Il s’agit d’un problème inférentiel difficile où les _a priori_ et les __mouvements__ qui sont utilisés peuvent avoir un impact important sur le résultat et sur la convergence du __MCMC__. Dans ce projet, on va ré-analyser un ensemble de données construit par _Inoue et al._ (cfr. __1.1 L'article__) pour évaluer la robustesse de leurs résultats par rapport aux _a priori_ ou aux __mouvements__ qui sont utilisés.

### 1.3 Les données
Comme anticipé dans la section __1.1 L'article__, les données sont constituées par des génomes mitochrondiaux entiers assemblés de 8 _holophaceans_ (répresentant les trois familles):
- _Callorhinchus milii_ (**_Callorhinchidae_**)
- _Callorhinchus capensis_ (**_Callorhinchidae_**)
- _Callorhinchus callorynchus_ (**_Callorhinchidae_**)
- _Hariotta raleighana_ (**_Rhinochimaeridae_**)
- _Rhinochimaera pacifica_ (**_Rhinochimaeridae_**)
- _Hydrolagus lemures_ (**_Chimaeridae_**)
- _Chimaera sp_ (**_Chimaeridae_**)
- _Chimaera monstrosa_ (**_Chimaeridae_**)

Et 15 autres vertebres: _Cyprinus carpio_, _Polymixia japonica_, _Tetraodon nigroviridis_, _Protopterus aethiopicus_, _Xenopus tropicalis_, _Struthio camelus_, _Ornithorhynchus anatinus_, _Homo sapiens_, _Plesiobatis daviesi_, _Okamejei kenojei_, _Amblyraja radiata_, _Squalus acanthias_, _Heterodontus francisci_, _Scyliorhinus canicula_, _Mustelus manazo_.
<br/>
Les séquences sont contenues dans le fichier `aln_prot.fst`. Il contient les séquences d'acides aminés de 12 gènes codant des protéines des génomes mitochondriaux (exclusion des gènes ARNt et ARNr).

?On a pas les deux outgroups
<br/>
?Les données sont des séquences d'aminoacides